In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et


In [2]:
xmlfiles=glob('./data_images/*.xml')
replace_text = lambda x: x.replace('\\','/')
xmlfiles = list(map(replace_text,xmlfiles))

In [3]:
xmlfiles

['./data_images/01c2d116b3.xml',
 './data_images/020773fcda.xml',
 './data_images/046c08cf-f51f-4f43-a580-95c599dd2064.xml',
 './data_images/04a53a216b.xml',
 './data_images/068edfbe-9daa-4ec0-bba0-953eaaf2566c.xml',
 './data_images/068f8bb738.xml',
 './data_images/06f74810f9.xml',
 './data_images/0b224df6-4fcf-41f2-a011-fbe453802e39.xml',
 './data_images/0c046787-a8b7-4322-8cde-66618f440297.xml',
 './data_images/0c3d0789f7.xml',
 './data_images/0c63f3b7c4.xml',
 './data_images/0daabbf9-0f43-4d1b-8e44-b76f28a42438.xml',
 './data_images/0ebc83cd-5c96-4d05-a3c8-5a8506f3db2b.xml',
 './data_images/0ee7d4a7-8ef3-413a-a2da-c7aaec716e76.xml',
 './data_images/0f12433f-e74f-48b9-8d74-b6acb6b976fe.xml',
 './data_images/1084bac3fb.xml',
 './data_images/1088e66ae4.xml',
 './data_images/13add66d67.xml',
 './data_images/144e7250-2bdc-44f7-827f-e619a6af5aba.xml',
 './data_images/14c840f203.xml',
 './data_images/14dbe5af8e.xml',
 './data_images/15608566-cf8e-45d6-9d12-ecf9999a9378.xml',
 './data_image

In [4]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    
    image_name = root.find('filename').text
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
        
    return parser

In [5]:
parser_all = list(map(extract_text, xmlfiles))

In [6]:
data = reduce(lambda x,y :x+y,parser_all)

In [7]:
df = pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [8]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,01c2d116b3.jpg,1600,1200,Deer,391,1369,156,1200
1,020773fcda.jpg,2832,4256,Deer,359,2832,532,3528
2,046c08cf-f51f-4f43-a580-95c599dd2064.jpg,748,421,Tiger,58,579,11,421
3,04a53a216b.jpg,3663,2346,Deer,273,3006,180,2321
4,068edfbe-9daa-4ec0-bba0-953eaaf2566c.jpg,1189,793,Tiger,47,1186,4,793


In [9]:
df.shape

(2142, 8)

In [10]:
df['name'].value_counts()

name
poacher       504
Elephant      341
ranger        288
gun           231
Rhinoceros    174
Tiger         158
Deer          119
rhino          70
elephant       61
deer           60
backpack       50
dog            42
tiger          33
wolf            4
bag             3
umbrella        2
car             1
motorcycle      1
Name: count, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2142 non-null   object
 1   width     2142 non-null   object
 2   height    2142 non-null   object
 3   name      2142 non-null   object
 4   xmin      2142 non-null   object
 5   xmax      2142 non-null   object
 6   ymin      2142 non-null   object
 7   ymax      2142 non-null   object
dtypes: object(8)
memory usage: 134.0+ KB


In [12]:
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2142 non-null   object
 1   width     2142 non-null   int64 
 2   height    2142 non-null   int64 
 3   name      2142 non-null   object
 4   xmin      2142 non-null   int64 
 5   xmax      2142 non-null   int64 
 6   ymin      2142 non-null   int64 
 7   ymax      2142 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 134.0+ KB


In [25]:
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
df['w'] = (df['xmax']-df['xmin'])/df['width']
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [26]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,01c2d116b3.jpg,1600,1200,Deer,391,1369,156,1200,0.550000,0.565000,0.611250,0.870000
1,020773fcda.jpg,2832,4256,Deer,359,2832,532,3528,0.563383,0.476974,0.873234,0.703947
2,046c08cf-f51f-4f43-a580-95c599dd2064.jpg,748,421,Tiger,58,579,11,421,0.425802,0.513064,0.696524,0.973872
3,04a53a216b.jpg,3663,2346,Deer,273,3006,180,2321,0.447584,0.533035,0.746110,0.912617
4,068edfbe-9daa-4ec0-bba0-953eaaf2566c.jpg,1189,793,Tiger,47,1186,4,793,0.518503,0.502522,0.957948,0.994956


In [27]:
images = df['filename'].unique()

In [28]:
len(images)

1090

In [29]:
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])

In [30]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [31]:
len(img_train), len(img_test)

(872, 218)

In [32]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [33]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,01c2d116b3.jpg,1600,1200,Deer,391,1369,156,1200,0.550000,0.565000,0.611250,0.870000
1,020773fcda.jpg,2832,4256,Deer,359,2832,532,3528,0.563383,0.476974,0.873234,0.703947
2,046c08cf-f51f-4f43-a580-95c599dd2064.jpg,748,421,Tiger,58,579,11,421,0.425802,0.513064,0.696524,0.973872
3,04a53a216b.jpg,3663,2346,Deer,273,3006,180,2321,0.447584,0.533035,0.746110,0.912617
4,068edfbe-9daa-4ec0-bba0-953eaaf2566c.jpg,1189,793,Tiger,47,1186,4,793,0.518503,0.502522,0.957948,0.994956


In [34]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
6,06f74810f9.jpg,1000,701,Deer,150,622,50,653,0.386000,0.501427,0.472000,0.860200
8,0c046787-a8b7-4322-8cde-66618f440297.jpg,2000,1333,Tiger,301,1591,50,1333,0.473000,0.518755,0.645000,0.962491
10,0c63f3b7c4.jpg,240,479,Deer,21,240,15,466,0.543750,0.502088,0.912500,0.941545
11,0daabbf9-0f43-4d1b-8e44-b76f28a42438.jpg,1920,1200,Tiger,341,1898,51,1183,0.583073,0.514167,0.810937,0.943333
20,14c840f203.jpg,432,301,Deer,38,414,2,301,0.523148,0.503322,0.870370,0.993355


In [37]:
def label_encoding(x):
    labels = {'Tiger':0, 'Elephant':1, 'Deer':2 , 'Rhinoceros':3, 'poacher':4, 'ranger':5, 'gun':6,'rhino':7,'elephant':8,'deer':9,'backpack':10,'tiger':11,'wolf':12,'bag':13, 'umbrella':14, 'car':15, 'motorcycle':16,'dog':17}
    return labels[x]

In [38]:
 train_df.loc[:, 'id'] = train_df['name'].apply(label_encoding)
 test_df.loc[:, 'id'] = test_df['name'].apply(label_encoding)
# train_df['id'] = train_df['name'].apply(label_encoding)
# test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\HP\AppData\Local\Temp\ipykernel_15608\775548244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, 'id'] = train_df['name'].apply(label_encoding)
C:\Users\HP\AppData\Local\Temp\ipykernel_15608\775548244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'id'] = test_df['name'].apply(label_encoding)


In [39]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,01c2d116b3.jpg,1600,1200,Deer,391,1369,156,1200,0.550000,0.565000,0.611250,0.870000,2
1,020773fcda.jpg,2832,4256,Deer,359,2832,532,3528,0.563383,0.476974,0.873234,0.703947,2
2,046c08cf-f51f-4f43-a580-95c599dd2064.jpg,748,421,Tiger,58,579,11,421,0.425802,0.513064,0.696524,0.973872,0
3,04a53a216b.jpg,3663,2346,Deer,273,3006,180,2321,0.447584,0.533035,0.746110,0.912617,2
4,068edfbe-9daa-4ec0-bba0-953eaaf2566c.jpg,1189,793,Tiger,47,1186,4,793,0.518503,0.502522,0.957948,0.994956,0
5,068f8bb738.jpg,680,390,Deer,286,632,22,390,0.675000,0.528205,0.508824,0.943590,2
7,0b224df6-4fcf-41f2-a011-fbe453802e39.jpg,1024,768,Tiger,102,1024,1,768,0.549805,0.500651,0.900391,0.998698,0
9,0c3d0789f7.jpg,426,280,Deer,139,398,20,236,0.630282,0.457143,0.607981,0.771429,2
12,0ebc83cd-5c96-4d05-a3c8-5a8506f3db2b.jpg,1920,1080,Tiger,1100,1920,20,1080,0.786458,0.509259,0.427083,0.981481,0
13,0ebc83cd-5c96-4d05-a3c8-5a8506f3db2b.jpg,1920,1080,Tiger,523,1097,24,1080,0.421875,0.511111,0.298958,0.977778,0


In [40]:
import os
from shutil import move

In [41]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [42]:
cols = ['filename','id','center_x','center_y','w','h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [43]:
def save_data(filename, folder_path, group_obj):
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path, filename)
    if os.path.exists(src):
        move(src,dst)

    text_filename = os.path.join(folder_path, os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)

In [44]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [45]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0      None
1      None
2      None
3      None
4      None
       ... 
867    None
868    None
869    None
870    None
871    None
Length: 872, dtype: object

In [46]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0      None
1      None
2      None
3      None
4      None
       ... 
213    None
214    None
215    None
216    None
217    None
Length: 218, dtype: object